In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv
/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv


In [3]:
data_train = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding = 'latin_1')

In [4]:
data_test= pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='latin_1')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re , string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


In [5]:
data_train.drop_duplicates(inplace=True)
data_test.drop_duplicates(inplace=True)

In [6]:
data_test.dropna(inplace=True)
data_train.dropna(inplace=True)

In [7]:
def change_sent(sentiment):
    if sentiment == "Extremly Positive":
        return "Positive"
    elif sentiment == "Extremely Negative":
        return "Negative Sentiment"
    elif sentiment == "Positive":
        return "Positive"
    elif sentiment == "Negative Sentiment":
        return "Negative Sentiment"
    else:
        return 'Neutral'
    

In [8]:
data_train['Sentiment']=data_train['Sentiment'].apply(lambda x:change_sent(x))
data_test['Sentiment']=data_test['Sentiment'].apply(lambda x:change_sent(x))

In [9]:
stop_word = stopwords.words('english')

In [10]:
def text_clean(text):
    text = re.sub(r'http\S+'," ",text)#remove url
    text = re.sub(r'@\w+',' ',text)#remove mentions
    text = re.sub(r'#\w+',' ',text)#remove hastags
    text = re.sub(r'\d+',' ' , text)#remove degits
    text = re.sub('r<.*?>',' ',text)#remove tags
    
    text= text.split()
    text = " ".join([word for word in text if not word in stop_word])
    
    return text

In [11]:
data_train['OriginalTweet']=data_train["OriginalTweet"].apply(lambda x:text_clean(x))
data_test['OriginalTweet']=data_test["OriginalTweet"].apply(lambda x:text_clean(x))

In [12]:
train_df = data_train.iloc[:,4:]
test_df = data_test.iloc[:,4:]

In [13]:
l = {"Neutral":0, "Positive":1,"Negative Sentiment":2}

In [14]:
train_df['Sentiment']=train_df['Sentiment'].map(l)
test_df['Sentiment']=test_df['Sentiment'].map(l)

In [15]:
x_train = train_df['OriginalTweet'].copy()
y_train = train_df['Sentiment'].copy()

x_test = test_df['OriginalTweet'].copy()
y_test = test_df['Sentiment'].copy()

In [16]:
max_len = np.max(x_train.apply(lambda x : len(x)))

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
vocab_length = len(tokenizer.word_index)+1


In [18]:
x_train=tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [19]:
x_train=pad_sequences(x_train,maxlen=max_len,padding='post')
x_test = pad_sequences(x_test,maxlen=max_len,padding='post')

In [20]:
embedding_dim = 16

In [21]:
model= tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_length,embedding_dim,input_length=max_len))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256,return_sequences=True)))

model.add(tf.keras.layers.GlobalAveragePooling1D())
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(3,activation='softmax'))
#opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy",optimizer ="Adam",metrics=["accuracy"])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 286, 16)           508144    
_________________________________________________________________
bidirectional (Bidirectional (None, 286, 512)          420864    
_________________________________________________________________
global_average_pooling1d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 962,035
Trainable params: 962,035
Non-trainable params: 0
__________________________________________________

In [23]:
y_test = to_categorical(y_test,3)
y_train = to_categorical(y_train,3)

In [24]:
history = model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test))

Epoch 1/10
1018/1018 [==============================] - 42s 41ms/step - loss: 0.9173 - accuracy: 0.5893 - val_loss: 0.8195 - val_accuracy: 0.6184
Epoch 2/10
1018/1018 [==============================] - 42s 41ms/step - loss: 0.7579 - accuracy: 0.6395 - val_loss: 0.7678 - val_accuracy: 0.6538
Epoch 3/10
1018/1018 [==============================] - 42s 41ms/step - loss: 0.6667 - accuracy: 0.6738 - val_loss: 0.6824 - val_accuracy: 0.6953
Epoch 4/10
1018/1018 [==============================] - 42s 41ms/step - loss: 0.5114 - accuracy: 0.7712 - val_loss: 0.6698 - val_accuracy: 0.7230
Epoch 5/10
1018/1018 [==============================] - 42s 41ms/step - loss: 0.4025 - accuracy: 0.8362 - val_loss: 0.7111 - val_accuracy: 0.7389
Epoch 6/10
1018/1018 [==============================] - 42s 41ms/step - loss: 0.3171 - accuracy: 0.8772 - val_loss: 0.7904 - val_accuracy: 0.7287
Epoch 7/10
1018/1018 [==============================] - 42s 42ms/step - loss: 0.2400 - accuracy: 0.9124 - val_loss: 0.9459 -

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [25]:
epochs = range(len(acc))

plt.plot(epochs, acc,'b',label='training acc')
plt.plot(epochs, val_acc, 'r', label='validation acc')
plt.legend()
plt.show()


plt.plot(epochs, loss,'b',label='training loss')
plt.plot(epochs, val_loss, 'r', label='validation loss')
plt.legend()
plt.show()

NameError: name 'acc' is not defined